# Analisar discursos

O objetivo desse notebook é analisar discursos disponíveis no diretório _data. Para preparar uma nova base, execute o notebook 01_preparar_base_discursos.ipynb

## Carregar dados

In [ ]:
import pandas as pd
from pathlib import Path

DATA = Path("_data/discursos_2019-02-01_2023-01-31.parquet")
df = pd.read_parquet(DATA)

df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
df = df[df["ok"].eq(True) & df["TextoDiscursoIntegral"].notna() & df["TextoDiscursoIntegral"].str.len().gt(0)]
df["ano"] = df["Data"].dt.year
df["mes"] = df["Data"].dt.to_period("M").astype(str)  # ex.: '2019-03'


## Mapa de temas (tópicos)

In [ ]:
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

def normalize(s):
    s = unidecode((s or "").lower())
    s = re.sub(r"[^a-z0-9áéíóúãõç\s]", " ", s)  # simples; ajuste se quiser
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["texto_clean"] = df["TextoDiscursoIntegral"].apply(normalize)

# embeddings (semântica)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
emb = model.encode(df["texto_clean"].tolist(), batch_size=64, show_progress_bar=True, normalize_embeddings=True)

# clusterização
k = 20  # ajuste depois de olhar coerência
km = KMeans(n_clusters=k, n_init=10, random_state=42)
df["topico"] = km.fit_predict(emb)

# palavras-chave por tópico via TF-IDF médio (c-TF-IDF simplificado)
vec = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.8)
X = vec.fit_transform(df["texto_clean"])
vocab = np.array(vec.get_feature_names_out())

def top_terms_for_topic(t, n=12):
    idx = np.where(df["topico"].values == t)[0]
    if len(idx)==0: return []
    mean_tfidf = X[idx].mean(axis=0).A1
    return vocab[mean_tfidf.argsort()[::-1][:n]].tolist()

termos = {t: top_terms_for_topic(t) for t in sorted(df["topico"].unique())}
pd.DataFrame({"topico": list(termos.keys()), "termos": list(termos.values())}).head()


### Detalhes de um Parlamentar

In [ ]:

codigos_parlamentares = dataframe_senadores_lista_legislatura["/Servico/Parlamentares/Parlamentar/IdentificacaoParlamentar/CodigoParlamentar"]

#codigos_parlamentares

urls_detalhe_senador = [base_url + f"senador/{codigo_parlamentar}.csv" for codigo_parlamentar in codigos_parlamentares]
urls_detalhe_senador[0]
csv_detalhes_senador = [f"_data/senador_{codigo_parlamentar}.csv" for codigo_parlamentar in codigos_parlamentares]
csv_detalhes_senador[0]


requests.get(urls_detalhe_senador[0], params, headers=headers, stream=True, timeout=60)

for i in range(len(urls_detalhe_senador)):
    with requests.get(urls_detalhe_senador[i], headers=headers, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(csv_detalhes_senador[i], "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    print(f"Detalhes do Senador {urls_detalhe_senador[i]} gravado com sucesso em {csv_detalhes_senador[i]}")

#response = requests.get(url[0], headers=headers)

#detalhe_parlamentar = response.json()
#detalhe_parlamentar = detalhe_parlamentar["DetalheParlamentar"]["Parlamentar"]["IdentificacaoParlamentar"]
#detalhe_parlamentar = pd.json_normalize(detalhe_parlamentar, sep=".")

#detalhes_parlamentares
#detalhe_parlamentares = detalhe_parlamentar


#detalhe_parlamentares.append(detalhe_parlamentar)

#detalhe_parlamentares







## Entes

In [ ]:
# ================================
# 5. Estatísticas descritivas
# ================================
print("Total de proposições:", len(df))
print("\nResumo estatístico:")
print(df.describe(include="all"))


In [ ]:
# ================================
# 6. Visualizações iniciais
# ================================

# Quantidade de proposições por tipo
df["siglaTipo"].value_counts().plot(kind="bar", figsize=(8,4))
plt.title("Distribuição por Tipo de Proposição")
plt.xlabel("Tipo")
plt.ylabel("Quantidade")
plt.show()

# Proposições por ano
df["ano"].value_counts().sort_index().plot(kind="line", marker="o", figsize=(8,4))
plt.title("Proposições por Ano")
plt.xlabel("Ano")
plt.ylabel("Quantidade")
plt.show()


In [ ]:
# ================================
# 7. Discussão preliminar (anotações)
# ================================
# - Identificar variáveis com dados ausentes.
print(df.isna().sum())

# - Refletir sobre possíveis problemas:
#   * Situações pouco padronizadas.
#   * Textos longos em 'ementa' -> análise qualitativa ou NLP.
#   * Discrepâncias entre anos com muitas/ poucas proposições.


In [ ]:
df = pd.read_csv("_data/discursos_2019-01-01_2023-01-31.csv", sep=";")
df.head()